In [ ]:
""" Remarque:
    - on réutilise le même fichier netcdf que dans check_output_FLEXPART --> pourrait peut-être le garder d'un programme à l'autre ?
        --> SURTOUT si on veut utiliser argmax
         |-> mais peut-être pas la peine, jsp
            --> PBM : loop through chaque flight dans check output flexpart
                --> could write a new netcdf file avec juste les trucs qui nous sont utiles --> plus léger à reprendre pour comparaisons ?
    - on utilise des chunks et tout mais du coup faudrait pas initialiser un client dask ou jsp quoi ??
    - <!> lon et lat values en dur
    - faire des fonctions pour recup les stats chimiques parce que là c'est du copié-collé des mêmes opérations
    - même pour récups les données chimiques, aussi des copié-collés
    - pareil pour l'affichage des résultats, easy peasy avec une fonction
"""
from netCDF4 import Dataset
#from ncdump import ncdump
import numpy as np
import pandas as pd
import xarray as xr
import json

#for plotting
import hvplot.xarray # fancy plotting for xarray
import holoviews as hv
import matplotlib.pyplot as plt
import geoviews as gv # j'ai pas l'impression qu'on s'en serve de ce truc

#for stats
import statistics
from statistics import mean, median, mode, stdev, median_high

#for datetime
import datetime
from datetime import datetime as dt
import matplotlib.dates as mdates
from matplotlib.dates import date2num

#for cartopy
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.feature as cfeature

In [ ]:
ds = xr.open_dataset('/o3p/macc/flexpart10.4/flexpart_v10.4_3d7eebf/src/exercises/soft-io-li/flight_2018_001_1h_05deg/10j_100k_output/grid_time_20180603150000.nc')

In [ ]:
ds

In [ ]:
ds.spec001_mr.sizes

In [ ]:
da = ds.spec001_mr.chunk(
    {'nageclass': 1, 'pointspec': 1, 'time': 40, 'height': -1, 'latitude': -1, 'longitude': -1}
) # -1 --> no chunking along this dimension
# let's get rid of the auxilary dimension "nageclass", for the sake of clarity:
da = da.squeeze(dim='nageclass')
da #######

In [ ]:
#spec_argmax = da.argmax(dim=['longitude', 'latitude'])

In [ ]:
## let's grab longitudes and latitudes of max. value of spec001_mr:
#lon_argmax = spec_argmax['longitude']
#lat_argmax = spec_argmax['latitude']
## let's see what's in lon_argmax, for example
##lon_argmax

In [ ]:
## the data is not yet computed; in order to actually fire the computation (load data from netcdf and process it), we can call a method "load"; it will take a couple of seconds to execute:
#lon_argmax.load()
#lat_argmax.load()
## and let's see what's in lon_argmax, for example:
##lon_argmax

In [ ]:
#spec_max = da.isel(longitude=lon_argmax, latitude=lat_argmax)
## again, we have to actually load spec_max to see results:
#spec_max.load()

In [ ]:
#lon_argmax = lon_argmax.where(spec_max > 0)
#lat_argmax = lat_argmax.where(spec_max > 0)
#spec_max = spec_max.where(spec_max > 0)
##spec_max
#
#print(lon_argmax)

In [ ]:
da = da.assign_coords(pointspec=da.pointspec) # ajoute la variable pointspec (énumère les releases) en coordonnées
da
da.size

In [ ]:
da.dims

In [ ]:
da.coords

In [ ]:
type(da)

In [ ]:
print(da.latitude.values)

In [ ]:
print(len(da))

In [ ]:
da.where(da>0)

In [ ]:
da.where(da.pointspec==1)

In [ ]:
bogota = hv.Scatter((-74.1461, 4.7010)).opts(color='pink', size=3)

In [ ]:
frankfurt = hv.Scatter((8.5622, 50.0379)).opts(color='pink', size=3)

#### Recup valeurs de NOx du vol

In [ ]:
#db = xr.open_dataset('/o3p/macc/test/NOx_synched/NOx_IAGOS_timeseries_2018060302172202.nc')
db = xr.open_dataset('/o3p/macc/test/new_NOx_synched_L2/new_NOx_IAGOS_timeseries_2018060302172202.nc')

In [ ]:
db

#### Recup itinéraire du vol

In [ ]:
#flight_path = hv.Scatter((lons, lats)).opts(color='red', size=0.5, )
flight_path = hv.Scatter((db.lon, db.lat)).opts(color='red', size=0.25, )

#### Recup panaches détectés pendant étape 1

In [ ]:
lons_1 = []
lons_2 = []
lats_1 = []
lats_2 = []
""" <!> lon et lat en dur ici !!! valeurs proches mais PAS == valeurs indiquées dans Plume_info"""
for i in range(len(db.lon)):
    if db.lon[i] > -73.3314 and db.lon[i]< -66.6724 :
        lons_1.append(db.lon[i])
        lats_1.append(db.lat[i])
    """ 4 plumes detected mais FLEXPART a dit 2 pointspec DONC là on englobe les lon des 3 derniers panaches (sont proches) 
        --> du coup faut aussi pouvoir faire ça dans la part 3 ! regrouper des panaches détectés comme différents dans part 1 """
    if db.lon[i] > -40.1465 and db.lon[i]< -34.1894 : 
        lons_2.append(db.lon[i])
        lats_2.append(db.lat[i])
    
first_rel =  hv.Scatter((lons_1, lats_1)).opts(color='yellow', size=0.25, ) 
second_rel =  hv.Scatter((lons_2, lats_2)).opts(color='yellow', size=0.25, ) 

In [ ]:
########### test perso


#### Recup données statistiques chimiques NOx premier panache

In [ ]:
#Calculate median of NOx values in these regions
""" <!> pourrait faire ça en même temps qu'au dessus, c'est la même boucle """
NOx_1 = []
NOx_2 = []
for i in range(len(db.lon)):
    if db.lon[i] > -73.3314 and db.lon[i]< -66.6724 :
        if db.NOx_P2b[i]> 0:
            if db.NOx_P2b_val[i]== 0:
                NOx_1.append(db.NOx_P2b[i])
    if db.lon[i] > -40.1465 and db.lon[i]< -34.1894 :
        if db.NOx_P2b[i]> 0:
            if db.NOx_P2b_val[i]== 0:
                NOx_2.append(db.NOx_P2b[i])

In [ ]:
############
print(type(NOx_1[0]))
print(NOx_1[0])
print(float(NOx_1[0]))

In [ ]:
if len(NOx_1) > 1:
    """ <?> pourquoi NOx_1_list alors que NOx_1 est déja une liste ?? 
    ==> met en float !! no problemo du coup """
    NOx_1_list = [float(n) for n in NOx_1]
    mean_NOx_1 = statistics.mean(NOx_1_list) 
    std_NOx_1 = statistics.stdev(NOx_1_list)
    fifth_perc_NOx_1 = np.percentile(NOx_1_list, 5)
    nintyfifth_perc_NOx_1 = np.percentile(NOx_1_list, 95)
    NOx_1_list=sorted(NOx_1_list)
    median_NOx_1 = statistics.median(NOx_1_list)
    upper_NOx_1_list = []
    lower_NOx_1_list = []

    for i in range(len(NOx_1_list)):
        if NOx_1_list[i]>median_NOx_1:
            upper_NOx_1_list.append(NOx_1_list[i])
        if NOx_1_list[i]<median_NOx_1:
            lower_NOx_1_list.append(NOx_1_list[i])
        
    upper_quartile_NOx_1 = statistics.median(upper_NOx_1_list)
    lower_quartile_NOx_1 = statistics.median(lower_NOx_1_list)
#if flights_JUN_EU > 0:
print("Number of entries NOx excess 1 :",len(NOx_1))
print("             Mean NOx excess 1 :",mean_NOx_1)
print("            Stdev NOx excess 1 :",std_NOx_1)
print("           Median NOx excess 1 :",median_NOx_1)
print("               Q3 NOx excess 1 :",upper_quartile_NOx_1)
print("               Q1 NOx excess 1 :",lower_quartile_NOx_1)
print("              5th NOx excess 1 :",fifth_perc_NOx_1)
print("             95th NOx excess 1 :",nintyfifth_perc_NOx_1)

#### Recup données statistiques chimiques NOx deuxième panache

In [ ]:
if len(NOx_2) > 1:
    """ <?> pourquoi NOx_2_list alors que NOx_2 est déja une liste ?? """
    NOx_2_list = [float(n) for n in NOx_2]
    #print(NOx_2_list)
    mean_NOx_2 = statistics.mean(NOx_2_list) 
    std_NOx_2 = statistics.stdev(NOx_2_list)
    fifth_perc_NOx_2 = np.percentile(NOx_2_list, 5)
    nintyfifth_perc_NOx_2 = np.percentile(NOx_2_list, 95)
    NOx_2_list=sorted(NOx_2_list)
    median_NOx_2 = statistics.median(NOx_2_list)
    upper_NOx_2_list = []
    lower_NOx_2_list = []

    for i in range(len(NOx_2_list)):
        if NOx_2_list[i]>median_NOx_2:
            upper_NOx_2_list.append(NOx_2_list[i])
        if NOx_2_list[i]<median_NOx_2:
            lower_NOx_2_list.append(NOx_2_list[i])
        
    upper_quartile_NOx_2 = statistics.median(upper_NOx_2_list)
    lower_quartile_NOx_2 = statistics.median(lower_NOx_2_list)
#if flights_JUN_EU > 0:
print("Number of entries NOx excess 2 :",len(NOx_2))
print("             Mean NOx excess 2 :",mean_NOx_2)
print("            Stdev NOx excess 2 :",std_NOx_2)
print("           Median NOx excess 2 :",median_NOx_2)
print("               Q3 NOx excess 2 :",upper_quartile_NOx_2)
print("               Q1 NOx excess 2 :",lower_quartile_NOx_2)
print("              5th NOx excess 2 :",fifth_perc_NOx_2)
print("             95th NOx excess 2 :",nintyfifth_perc_NOx_2)

#### Recup données statistiques chimiques CO

In [ ]:
#Calculate median of CO values in these regions
CO_1 = []
CO_2 = []

for i in range(len(db.lon)):
    if db.lon[i] > -73.3314 and db.lon[i]< -66.6724 :
        if db.CO_P1[i]> 0:
            #if db.NOx_P2b_val[i]== 0:
                CO_1.append(db.CO_P1[i])
    if db.lon[i] > -40.1465 and db.lon[i]< -34.1894 :
        if db.CO_P1[i]> 0:
            #if db.NOx_P2b_val[i]== 0:
                CO_2.append(db.CO_P1[i])
                

In [ ]:
if len(CO_1) > 1:
    CO_1_list = [float(n) for n in CO_1]
    mean_CO_1 = statistics.mean(CO_1_list) 
    std_CO_1 = statistics.stdev(CO_1_list)
    fifth_perc_CO_1 = np.percentile(CO_1_list, 5)
    nintyfifth_perc_CO_1 = np.percentile(CO_1_list, 95)
    CO_1_list=sorted(CO_1_list)
    median_CO_1 = statistics.median(CO_1_list)
    upper_CO_1_list = []
    lower_CO_1_list = []

    for i in range(len(CO_1_list)):
        if CO_1_list[i]>median_CO_1:
            upper_CO_1_list.append(CO_1_list[i])
        if CO_1_list[i]<median_CO_1:
            lower_CO_1_list.append(CO_1_list[i])
        
    upper_quartile_CO_1 = statistics.median(upper_CO_1_list)
    lower_quartile_CO_1 = statistics.median(lower_CO_1_list)
#if flights_JUN_EU > 0:
print("Number of entries CO excess 1 :",len(CO_1))
print("             Mean CO excess 1 :",mean_CO_1)
print("            Stdev CO excess 1 :",std_CO_1)
print("           Median CO excess 1 :",median_CO_1)
print("               Q3 CO excess 1 :",upper_quartile_CO_1)
print("               Q1 CO excess 1 :",lower_quartile_CO_1)
print("              5th CO excess 1 :",fifth_perc_CO_1)
print("             95th CO excess 1 :",nintyfifth_perc_CO_1)

In [ ]:
if len(CO_2) > 1:
    CO_2_list = [float(n) for n in CO_2]
    mean_CO_2 = statistics.mean(CO_2_list) 
    std_CO_2 = statistics.stdev(CO_2_list)
    fifth_perc_CO_2 = np.percentile(CO_2_list, 5)
    nintyfifth_perc_CO_2 = np.percentile(CO_2_list, 95)
    CO_2_list=sorted(CO_2_list)
    median_CO_2 = statistics.median(CO_2_list)
    upper_CO_2_list = []
    lower_CO_2_list = []

    for i in range(len(CO_2_list)):
        if CO_2_list[i]>median_CO_2:
            upper_CO_2_list.append(CO_2_list[i])
        if CO_2_list[i]<median_CO_2:
            lower_CO_2_list.append(CO_2_list[i])
        
    upper_quartile_CO_2 = statistics.median(upper_CO_2_list)
    lower_quartile_CO_2 = statistics.median(lower_CO_2_list)
#if flights_JUN_EU > 0:
print("Number of entries CO excess 2 :",len(CO_2))
print("             Mean CO excess 2 :",mean_CO_2)
print("            Stdev CO excess 2 :",std_CO_2)
print("           Median CO excess 2 :",median_CO_2)
print("               Q3 CO excess 2 :",upper_quartile_CO_2)
print("               Q1 CO excess 2 :",lower_quartile_CO_2)
print("              5th CO excess 2 :",fifth_perc_CO_2)
print("             95th CO excess 2 :",nintyfifth_perc_CO_2)

#### Recup données statistiques chimiques O3

In [ ]:
#Calculate median of NOx values in these regions
O3_1 = []
O3_2 = []

for i in range(len(db.lon)):
    if db.lon[i] > -73.3314 and db.lon[i]< -66.6724 :
        if db.O3_P1[i]> 0:
            #if db.NOx_P2b_val[i]== 0:
                O3_1.append(db.O3_P1[i])
    if db.lon[i] > -40.1465 and db.lon[i]< -34.1894 :
        if db.O3_P1[i]> 0:
            #if db.NOx_P2b_val[i]== 0:
                O3_2.append(db.O3_P1[i])
                

In [ ]:
if len(O3_1) > 1:
    O3_1_list = [float(n) for n in O3_1]
    mean_O3_1 = statistics.mean(O3_1_list) 
    std_O3_1 = statistics.stdev(O3_1_list)
    fifth_perc_O3_1 = np.percentile(O3_1_list, 5)
    nintyfifth_perc_O3_1 = np.percentile(O3_1_list, 95)
    O3_1_list=sorted(O3_1_list)
    median_O3_1 = statistics.median(O3_1_list)
    upper_O3_1_list = []
    lower_O3_1_list = []

    for i in range(len(O3_1_list)):
        if O3_1_list[i]>median_O3_1:
            upper_O3_1_list.append(O3_1_list[i])
        if O3_1_list[i]<median_O3_1:
            lower_O3_1_list.append(O3_1_list[i])
        
    upper_quartile_O3_1 = statistics.median(upper_O3_1_list)
    lower_quartile_O3_1 = statistics.median(lower_O3_1_list)
#if flights_JUN_EU > 0:
print("Number of entries O3 excess 1 :",len(O3_1))
print("             Mean O3 excess 1 :",mean_O3_1)
print("            Stdev O3 excess 1 :",std_O3_1)
print("           Median O3 excess 1 :",median_O3_1)
print("               Q3 O3 excess 1 :",upper_quartile_O3_1)
print("               Q1 O3 excess 1 :",lower_quartile_O3_1)
print("              5th O3 excess 1 :",fifth_perc_O3_1)
print("             95th O3 excess 1 :",nintyfifth_perc_O3_1)

In [ ]:
if len(O3_2) > 1:
    O3_2_list = [float(n) for n in O3_2]
    mean_O3_2 = statistics.mean(O3_2_list) 
    std_O3_2 = statistics.stdev(O3_2_list)
    fifth_perc_O3_2 = np.percentile(O3_2_list, 5)
    nintyfifth_perc_O3_2 = np.percentile(O3_2_list, 95)
    O3_2_list=sorted(O3_2_list)
    median_O3_2 = statistics.median(O3_2_list)
    upper_O3_2_list = []
    lower_O3_2_list = []

    for i in range(len(O3_2_list)):
        if O3_2_list[i]>median_O3_2:
            upper_O3_2_list.append(O3_2_list[i])
        if O3_2_list[i]<median_O3_2:
            lower_O3_2_list.append(O3_2_list[i])
        
    upper_quartile_O3_2 = statistics.median(upper_O3_2_list)
    lower_quartile_O3_2 = statistics.median(lower_O3_2_list)
#if flights_JUN_EU > 0:
print("Number of entries O3 excess 2 :",len(O3_2))
print("             Mean O3 excess 2 :",mean_O3_2)
print("            Stdev O3 excess 2 :",std_O3_2)
print("           Median O3 excess 2 :",median_O3_2)
print("               Q3 O3 excess 2 :",upper_quartile_O3_2)
print("               Q1 O3 excess 2 :",lower_quartile_O3_2)
print("              5th O3 excess 2 :",fifth_perc_O3_2)
print("             95th O3 excess 2 :",nintyfifth_perc_O3_2)

#### Recup données statistiques nuages

In [ ]:
#Calculate statistics of Cloud values in these regions

Cloud_1 = []
Cloud_2 = []

Cloud_Pres_1 = 0
Cloud_Pres_2 = 0

for i in range(len(db.lon)):
    if db.lon[i] > -73.3314 and db.lon[i]< -66.6724 :
        Cloud_Pres_1 = Cloud_Pres_1 + db.cloud_presence_P1[i]
        if db.cloud_P1[i]> 0:
            print("yes!")
            if db.cloud_P1_val[i]== 0:
                print("yes!")
                Cloud_1.append(db.cloud_P1[i])
    if db.lon[i] > -40.1465 and db.lon[i]< -34.1894 :
        Cloud_Pres_2 = Cloud_Pres_2 + db.cloud_presence_P1[i]
        if db.cloud_P1[i]> 0:
            if db.cloud_P1_val[i]== 0:
                Cloud_2.append(db.NOx_P2b[i])

In [ ]:
if Cloud_Pres_1 > 0:
    print("Cloud presence indicated")
else:
    print("Cloud presence NOT indicated")
""" mettre ce if dans cloud_pre_1 > 0 OK non? sert à rien qu'il soit là si sait déjà que <= 0"""
if len(Cloud_1) >= 1:
    Cloud_1_list = [float(n) for n in Cloud_1]
    mean_Cloud_1 = statistics.mean(Cloud_1_list) 
    std_Cloud_1 = statistics.stdev(Cloud_1_list)
    fifth_perc_Cloud_1 = np.percentile(Cloud_1_list, 5)
    nintyfifth_perc_Cloud_1 = np.percentile(Cloud_1_list, 95)
    Cloud_1_list=sorted(Cloud_1_list)
    median_Cloud_1 = statistics.median(Cloud_1_list)
    upper_Cloud_1_list = []
    lower_Cloud_1_list = []

    for i in range(len(Cloud_1_list)):
        if Cloud_1_list[i]>median_Cloud_1:
            upper_Cloud_1_list.append(Cloud_1_list[i])
        if Cloud_1_list[i]<median_Cloud_1:
            lower_Cloud_1_list.append(Cloud_1_list[i])
        
    upper_quartile_Cloud_1 = statistics.median(upper_Cloud_1_list)
    lower_quartile_Cloud_1 = statistics.median(lower_Cloud_1_list)
if len(Cloud_1) >= 1:
    print("Number of entries Cloud 1 :",len(Cloud_1))
    print("             Mean Cloud 1 :",mean_Cloud_1)
    print("            Stdev Cloud 1 :",std_Cloud_1)
    print("           Median Cloud 1 :",median_Cloud_1)
    print("               Q3 Cloud 1 :",upper_quartile_Cloud_1)
    print("               Q1 Cloud 1 :",lower_quartile_Cloud_1)
    print("              5th Cloud 1 :",fifth_perc_Cloud_1)
    print("             95th Cloud 1 :",nintyfifth_perc_Cloud_1)
else:
    print("No cloud data available")

In [ ]:
if Cloud_Pres_2 > 0:
    print("Cloud presence indicated")
else:
    print("Cloud presence NOT indicated")

if len(Cloud_2) >= 1:
    Cloud_2_list = [float(n) for n in Cloud_2]
    mean_Cloud_2 = statistics.mean(Cloud_2_list) 
    std_Cloud_2 = statistics.stdev(Cloud_2_list)
    fifth_perc_Cloud_2 = np.percentile(Cloud_2_list, 5)
    nintyfifth_perc_Cloud_2 = np.percentile(Cloud_2_list, 95)
    Cloud_2_list=sorted(Cloud_2_list)
    median_Cloud_2 = statistics.median(Cloud_2_list)
    upper_Cloud_2_list = []
    lower_Cloud_2_list = []

    for i in range(len(Cloud_2_list)):
        if Cloud_2_list[i]>median_Cloud_2:
            upper_Cloud_2_list.append(Cloud_2_list[i])
        if Cloud_2_list[i]<median_Cloud_2:
            lower_Cloud_2_list.append(Cloud_2_list[i])
        
    upper_quartile_Cloud_2 = statistics.median(upper_Cloud_2_list)
    lower_quartile_Cloud_2 = statistics.median(lower_Cloud_2_list)
if len(Cloud_2) >= 1:
    print("Number of entries Cloud 2 :",len(Cloud_2))
    print("             Mean Cloud 2 :",mean_Cloud_2)
    print("            Stdev Cloud 2 :",std_Cloud_2)
    print("           Median Cloud 2 :",median_Cloud_2)
    print("               Q3 Cloud 2 :",upper_quartile_Cloud_2)
    print("               Q1 Cloud 2 :",lower_quartile_Cloud_2)
    print("              5th Cloud 2 :",fifth_perc_Cloud_2)
    print("             95th Cloud 2 :",nintyfifth_perc_Cloud_2)
else:
    print("No cloud data available")
    

In [ ]:
#Calculate statistics for RHL values in these regions

#Data not yet available! Normally at the end of November.

#RHL_1 = []
#RHL_2 = []

#for i in range(len(db.lon)):
#    if db.lon[i] > -73.3314 and db.lon[i]< -66.6724 :
#        if db.RHL_P1[i]> 0:
#            if db.RHL_P1_val[i]== 0:
#                RHL_1.append(db.RHL_P1[i])
#    if db.lon[i] > -40.1465 and db.lon[i]< -34.1894 :
#        if db.RHL_P1[i]> 0:
#            if db.RHL_P1_val[i]== 0:
#                RHL_2.append(db.RHL_P2b[i])

In [ ]:
#Results of RHL data, need to see it to understand what's required here.

### Carte avec flight path, releases and plume dispersion over 10 days (sum sur height et pointspec)

In [ ]:
da_all_agregated = da.sum(dim=('height', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='FLEXPART output: plume dispersion over 10 days',
    xlabel='Longitude (°)', ylabel='Latitude (°)', clabel='Res. Time (s)', 
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, global_extent=True,
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * flight_path * first_rel * second_rel * bogota * frankfurt


In [ ]:
da_all_agregated = da.sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='FLEXPART output: plume dispersion over 10 days',
    xlabel='Longitude (°)', ylabel='Latitude (°)', clabel='Res. Time (s)', 
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, global_extent=True,
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * flight_path * first_rel * second_rel * bogota * frankfurt


In [ ]:
da_all_agregated = da.sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='FLEXPART output: plume dispersion over 10 days',
    xlabel='Longitude (°)', ylabel='Latitude (°)', clabel='Res. Time (s)', 
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-180, 0), ylim=(-60, 60),
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
""" en fait ça affiche que pl, ça ajoute pas dessus le flight_path ni le reste """
""" si en fait, jsp pourquoi c'est ok maintenant """
plot = (pl * flight_path  * bogota * frankfurt * first_rel * second_rel)
plot

In [ ]:
da_all_agregated = da.isel(pointspec=0).sum(dim=('height', 'time'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='Max residence time for all times and altitudes',
    xlabel='Longitude (°)', ylabel='Latitude (°)', clabel='Res. Time (s)', 
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, global_extent=True,
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * flight_path  * bogota * frankfurt * first_rel # * second_rel

In [ ]:
da_all_agregated = da.isel(pointspec=1).sum(dim=('height', 'time'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='Plume dispersion over 10 days',
    xlabel='Longitude (°)', ylabel='Latitude (°)', clabel='Res. Time (s)', 
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, global_extent=True,
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * flight_path  * bogota * frankfurt * first_rel * second_rel

In [ ]:
da_time_agregated = da.sum(dim='time')
pl = da_time_agregated.hvplot.image(
    x='longitude', y='latitude', groupby=['height', 'pointspec'], 
    xlabel='Longitude (°)', ylabel='Latitude (°)', clabel='Res. Time (s)', 
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, global_extent=True,
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * flight_path  * bogota * frankfurt * first_rel * second_rel

In [ ]:
da_all_agregated = da.isel(pointspec=0).sum(dim=('height', 'time'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='Max residence time for all times and altitudes', 
    xlabel='Longitude (°)', ylabel='Latitude (°)', clabel='Res. Time (s)', 
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, global_extent=True,
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * flight_path  * bogota * frankfurt * first_rel * second_rel

In [ ]:
# Repeat for various heights 8-13km
da_all_agregated = da.isel(height=slice(15,25),pointspec=0).sum(dim=('height', 'time'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude', title='Max residence time for all times and 8-13 km altitude',
    xlabel='Longitude (°)', ylabel='Latitude (°)', clabel='Res. Time (s)', 
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-150, 30), ylim=(-60, 60),
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * flight_path  * bogota * frankfurt * first_rel * second_rel

In [ ]:
# Repeat for various heights 2 - 8km
da_all_agregated = da.isel(height=slice(3,15),pointspec=0).sum(dim=('height', 'time'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude', title='FLEXPART output: plume dispersion over 10 days, 2-8km',
    xlabel='Longitude (°)', ylabel='Latitude (°)', clabel='Res. Time (s)', 
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-150, 30), ylim=(-60, 60),
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * flight_path  * bogota * frankfurt * first_rel * second_rel

In [ ]:
# Repeat for various heights 0 - 2km
da_all_agregated = da.isel(height=slice(0,3),pointspec=0).sum(dim=('height', 'time'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude', title='FLEXPART output: plume dispersion over 10 days, 0-2km',
    xlabel='Longitude (°)', ylabel='Latitude (°)', clabel='Res. Time (s)', 
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-150, 30), ylim=(-60, 60),
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * flight_path  * bogota * frankfurt * first_rel * second_rel

In [ ]:
#da_all_agregated = da.isel(pointspec=0).sum(dim=('height', 'time'))
da_all_agregated = da.sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude', title='FLEXPART output: plume dispersion over 10 days',
    xlabel='Longitude (°)', ylabel='Latitude (°)', clabel='Res. Time (s)', 
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, global_extent=True,
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * flight_path  * bogota * frankfurt * first_rel * second_rel

---

### petit test perso pour voir la diff quand sum(dim=('height', 'time', 'pointspec')) et sum(dim=('height', 'time'))

In [ ]:
#da_all_agregated = da.isel(pointspec=0).sum(dim=('height', 'time'))
da_all_agregated = da.sum(dim=('height', 'time'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude', title='FLEXPART output: plume dispersion over 10 days',
    xlabel='Longitude (°)', ylabel='Latitude (°)', clabel='Res. Time (s)', 
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, global_extent=True,
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * flight_path  * bogota * frankfurt * first_rel * second_rel

In [ ]:
type(da.isel(pointspec=1).sum(dim=('height')))

---

In [ ]:
da_all_agregated = da.isel(time=slice(11,12)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='Max residence time t-11h all altitudes',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Max Residence Time (s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_154_05deg/GLM_array_154_04-05_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)
#pl2 = (flash_en*100000000000).hvplot.image(
pl11 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl11 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl11

In [ ]:
da_all_agregated = da.isel(time=slice(12,13)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='Max residence time t-12h all altitudes',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Max Residence Time (s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_154_05deg/GLM_array_154_03-04_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)
#pl2 = (flash_en*100000000000).hvplot.image(
pl12 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl12 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl12

In [ ]:
da_all_agregated = da.isel(time=slice(13,14)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='Max residence time t-13h all altitudes',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Max Residence Time (s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_154_05deg/GLM_array_154_02-03_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)
#pl2 = (flash_en*100000000000).hvplot.image(
pl13 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl13 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl13

In [ ]:
da_all_agregated = da.isel(time=slice(14,15)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='Max residence time t-14h all altitudes',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Max Residence Time (s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_154_05deg/GLM_array_154_01-02_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)
#pl2 = (flash_en*100000000000).hvplot.image(
pl14 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl14 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl14

In [ ]:
da_all_agregated = da.isel(time=slice(15,16)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='Max residence time t-15h all altitudes',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Max Residence Time (s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_154_05deg/GLM_array_154_00-01_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)
#pl2 = (flash_en*100000000000).hvplot.image(
pl15 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl15 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl15

In [ ]:
da_all_agregated = da.isel(time=slice(16,17)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='Max residence time t-16h all altitudes',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Max Residence Time (s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_153_05deg/GLM_array_153_23-00_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)
#pl2 = (flash_en*100000000000).hvplot.image(
pl16 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl16 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl16

In [ ]:
# Hourly views from -24 hrs to -48 hrs for release 1
# Hour is -24 == 24/05/2015 @ 21h00
da_all_agregated = da.isel(time=slice(17,18)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='Zoom: Plume dispersion over 1hr + GLM output',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_153_05deg/GLM_array_153_22-23_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)
#pl2 = (flash_en*100000000000).hvplot.image(
pl17 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl17 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl17 * bogota * flight_path * first_rel * second_rel #plots them one on top of the other

In [ ]:
da_all_agregated = da.isel(time=slice(18,19)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-18h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_153_05deg/GLM_array_153_21-22_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl18 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl18 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl18

In [ ]:
da_all_agregated = da.isel(time=slice(19,20)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-19h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_153_05deg/GLM_array_153_20-21_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl19 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl19 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl19

In [ ]:
da_all_agregated = da.isel(time=slice(20,21)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-20h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_153_05deg/GLM_array_153_19-20_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl20 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl20 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl20

In [ ]:
da_all_agregated = da.isel(time=slice(21,22)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-21h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_153_05deg/GLM_array_153_18-19_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl21 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl21 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl21

In [ ]:
da_all_agregated = da.isel(time=slice(22,23)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-22h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_153_05deg/GLM_array_153_17-18_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl22 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl22 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl22

In [ ]:
da_all_agregated = da.isel(time=slice(23,24)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-23h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_153_05deg/GLM_array_153_16-17_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl23 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl23 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl23

In [ ]:
da_all_agregated = da.isel(time=slice(24,25)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-24h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_153_05deg/GLM_array_153_15-16_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl24 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl24 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl24

In [ ]:
da_all_agregated = da.isel(time=slice(25,26)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-25h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_153_05deg/GLM_array_153_14-15_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl25 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl25 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl25

In [ ]:
da_all_agregated = da.isel(time=slice(26,27)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-26h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_153_05deg/GLM_array_153_13-14_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl26 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl26 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl26 * bogota

In [ ]:
da_all_agregated = da.isel(time=slice(27,28)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-27h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_153_05deg/GLM_array_153_12-13_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl27 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl27 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl27

In [ ]:
da_all_agregated = da.isel(time=slice(28,29)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-28h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_153_05deg/GLM_array_153_11-12_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl28 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl28 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl28

In [ ]:
da_all_agregated = da.isel(time=slice(29,30)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-29h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_153_05deg/GLM_array_153_10-11_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl29 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl29 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl29

In [ ]:
da_all_agregated = da.isel(time=slice(30,31)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-30h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_153_05deg/GLM_array_153_09-10_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl30 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl30 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl30

In [ ]:
da_all_agregated = da.isel(time=slice(31,32)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-31h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_153_05deg/GLM_array_153_08-09_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl31 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl31 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl31

In [ ]:
da_all_agregated = da.isel(time=slice(32,33)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-32h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_153_05deg/GLM_array_153_07-08_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl32 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl32 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl32

In [ ]:
da_all_agregated = da.isel(time=slice(33,34)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-33h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_153_05deg/GLM_array_153_06-07_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl33 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl33 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl33

In [ ]:
da_all_agregated = da.isel(time=slice(34,35)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-34h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_153_05deg/GLM_array_153_05-06_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl34 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl34 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl34

In [ ]:
da_all_agregated = da.isel(time=slice(35,36)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-35h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_153_05deg/GLM_array_153_04-05_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl35 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl35 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl35

In [ ]:
da_all_agregated = da.isel(time=slice(36,37)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-36h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_153_05deg/GLM_array_153_03-04_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl36 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl36 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl36

In [ ]:
da_all_agregated = da.isel(time=slice(37,38)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-37h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_153_05deg/GLM_array_153_02-03_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl37 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl37 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl37

In [ ]:
da_all_agregated = da.isel(time=slice(38,39)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-38h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_153_05deg/GLM_array_153_01-02_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl38 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl38 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl38

In [ ]:
da_all_agregated = da.isel(time=slice(39,40)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-39h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_153_05deg/GLM_array_153_00-01_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl39 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl39 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl39

In [ ]:
da_all_agregated = da.isel(time=slice(40,41)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-40h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_152_05deg/GLM_array_152_23-00_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl40 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl40 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl40

In [ ]:
da_all_agregated = da.isel(time=slice(41,42)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-41h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_152_05deg/GLM_array_152_22-23_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl41 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl41 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl41

In [ ]:
da_all_agregated = da.isel(time=slice(42,43)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-42h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_152_05deg/GLM_array_152_21-22_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl42 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl42 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl42

In [ ]:
da_all_agregated = da.isel(time=slice(43,44)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-43h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_152_05deg/GLM_array_152_20-21_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl43 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl43 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl43

In [ ]:
da_all_agregated = da.isel(time=slice(44,45)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-44h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_152_05deg/GLM_array_152_19-20_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl44 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl44 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl44

In [ ]:
da_all_agregated = da.isel(time=slice(45,46)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-45h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
da_all_agregated = da.isel(time=slice(46,47)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-46h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_152_05deg/GLM_array_152_17-18_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl46 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl46 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl46

In [ ]:
da_all_agregated = da.isel(time=slice(47,48)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-47h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_152_05deg/GLM_array_152_16-17_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl47 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl47 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl47

In [ ]:
da_all_agregated = da.isel(time=slice(48,49)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-48h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_152_05deg/GLM_array_152_15-16_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl48 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl48 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl48

In [ ]:
da_all_agregated = da.isel(time=slice(40,48)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-40h --> t-49h ',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    #xlim=(-140, -20), ylim=(-60, 60),
    xlim=(-85, -45), ylim=(0, 60),
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl40 * pl41 * pl42 * pl43 * pl44 * pl46 * pl47 * pl48

In [ ]:
da_all_agregated = da.isel(time=slice(49,50)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-49h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_152_05deg/GLM_array_152_14-15_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl49 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl49 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl49

In [ ]:
da_all_agregated = da.isel(time=slice(50,51)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-50h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_152_05deg/GLM_array_152_13-14_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl50 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl50 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl50

In [ ]:
da_all_agregated = da.isel(time=slice(51,52)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-51h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_152_05deg/GLM_array_152_12-13_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl51 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl51 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl51

In [ ]:
da_all_agregated = da.isel(time=slice(52,53)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-52h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_152_05deg/GLM_array_152_11-12_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl52 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl52 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl52

In [ ]:
da_all_agregated = da.isel(time=slice(53,54)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-53h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_152_05deg/GLM_array_152_10-11_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl53 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl53 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl53

In [ ]:
da_all_agregated = da.isel(time=slice(54,55)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-54h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_152_05deg/GLM_array_152_09-10_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl54 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl54 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl54

In [ ]:
da_all_agregated = da.isel(time=slice(55,56)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-55h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_152_05deg/GLM_array_152_08-09_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl55 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl55 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl55

In [ ]:
da_all_agregated = da.isel(time=slice(56,57)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-56h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_152_05deg/GLM_array_152_07-08_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl56 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl56 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl56

In [ ]:
da_all_agregated = da.isel(time=slice(57,58)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-57h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_152_05deg/GLM_array_152_06-07_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl57 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl57 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl57

In [ ]:
da_all_agregated = da.isel(time=slice(58,59)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-58h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_152_05deg/GLM_array_152_05-06_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl58 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl58 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl58

In [ ]:
da_all_agregated = da.isel(time=slice(59,60)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-59h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_152_05deg/GLM_array_152_04-05_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl59 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl59 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl59

In [ ]:
da_all_agregated = da.isel(time=slice(60,61)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-60h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_152_05deg/GLM_array_152_03-04_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl60 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl60 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl60

In [ ]:
da_all_agregated = da.isel(time=slice(61,62)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-61h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_152_05deg/GLM_array_152_02-03_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl61 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl61 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl61

In [ ]:
da_all_agregated = da.isel(time=slice(62,63)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-62h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_152_05deg/GLM_array_152_01-02_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl62 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl62 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl62

In [ ]:
da_all_agregated = da.isel(time=slice(63,64)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-63h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_152_05deg/GLM_array_152_00-01_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl63 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl63 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl63

In [ ]:
da_all_agregated = da.isel(time=slice(64,65)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-64h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_151_05deg/GLM_array_151_23-00_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl64 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl64 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl64

In [ ]:
da_all_agregated = da.isel(time=slice(65,66)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-65h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_151_05deg/GLM_array_151_22-23_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl65 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl65 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl65

In [ ]:
da_all_agregated = da.isel(time=slice(66,67)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-66h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_151_05deg/GLM_array_151_21-22_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl66 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl66 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl66

In [ ]:
da_all_agregated = da.isel(time=slice(67,68)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-67h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_151_05deg/GLM_array_151_20-21_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl67 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl67 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl67

In [ ]:
da_all_agregated = da.isel(time=slice(68,69)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-68h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_151_05deg/GLM_array_151_19-20_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl68 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl68 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl68

In [ ]:
da_all_agregated = da.isel(time=slice(69,70)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-69h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
# No GLM for 18h-19h

In [ ]:
da_all_agregated = da.isel(time=slice(70,71)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-70h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_151_05deg/GLM_array_151_17-18_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl70 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl70 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl70

In [ ]:
da_all_agregated = da.isel(time=slice(71,72)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-71h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_151_05deg/GLM_array_151_16-17_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl71 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl71 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl71

In [ ]:
da_all_agregated = da.isel(time=slice(72,73)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-72h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_151_05deg/GLM_array_151_15-16_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl72 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl72 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl72

In [ ]:
da_all_agregated = da.isel(time=slice(73,74)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-73h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_151_05deg/GLM_array_151_14-15_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl73 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl73 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl73

In [ ]:
da_all_agregated = da.isel(time=slice(74,75)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-74h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_151_05deg/GLM_array_151_13-14_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl74 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl74 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl74

In [ ]:
da_all_agregated = da.isel(time=slice(75,76)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-75h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_151_05deg/GLM_array_151_12-13_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl75 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl75 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl75

In [ ]:
da_all_agregated = da.isel(time=slice(76,77)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-76h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_151_05deg/GLM_array_151_11-12_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl76 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl76 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl76

In [ ]:
da_all_agregated = da.isel(time=slice(77,78)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-77h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_151_05deg/GLM_array_151_10-11_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl77 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl77 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl77

In [ ]:
da_all_agregated = da.isel(time=slice(78,79)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-78h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_151_05deg/GLM_array_151_09-10_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl78 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl78 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl78

In [ ]:
da_all_agregated = da.isel(time=slice(79,80)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-79h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_151_05deg/GLM_array_151_08-09_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl79 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl79 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl79

In [ ]:
da_all_agregated = da.isel(time=slice(80,81)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-80h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_151_05deg/GLM_array_151_07-08_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl80 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl80 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl80

In [ ]:
da_all_agregated = da.isel(time=slice(81,82)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-81h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_151_05deg/GLM_array_151_06-07_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl81 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl81 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl81

In [ ]:
da_all_agregated = da.isel(time=slice(82,83)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-82h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_151_05deg/GLM_array_151_05-06_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl82 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl82 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl82

In [ ]:
da_all_agregated = da.isel(time=slice(83,84)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-83h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_151_05deg/GLM_array_151_04-05_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl83 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl83 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl83

In [ ]:
da_all_agregated = da.isel(time=slice(84,85)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-84h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_151_05deg/GLM_array_151_03-04_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl84 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl84 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl84

In [ ]:
da_all_agregated = da.isel(time=slice(85,86)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-85h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_151_05deg/GLM_array_151_02-03_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl85 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl85 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl85

In [ ]:
da_all_agregated = da.isel(time=slice(86,87)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-86h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_151_05deg/GLM_array_151_01-02_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl86 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl86 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl86

In [ ]:
da_all_agregated = da.isel(time=slice(87,88)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-87h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_151_05deg/GLM_array_151_00-01_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl87 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl87 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl87

In [ ]:
da_all_agregated = da.isel(time=slice(88,89)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-88h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_150_05deg/GLM_array_150_23-00_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl88 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl88 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl88

In [ ]:
da_all_agregated = da.isel(time=slice(89,90)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-89h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_150_05deg/GLM_array_150_22-23_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl89 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl89 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl89

In [ ]:
da_all_agregated = da.isel(time=slice(90,91)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-90h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_150_05deg/GLM_array_150_21-22_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl90 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl90 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl90

In [ ]:
da_all_agregated = da.isel(time=slice(91,92)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-91h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_150_05deg/GLM_array_150_20-21_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl91 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl91 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl91

In [ ]:
da_all_agregated = da.isel(time=slice(92,93)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-92h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
#GLM_array_150_19-20_batch_bis.nc does not exist

In [ ]:
#pl * pl92

In [ ]:
da_all_agregated = da.isel(time=slice(93,94)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-93h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_150_05deg/GLM_array_150_18-19_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl93 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl93 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl93

In [ ]:
da_all_agregated = da.isel(time=slice(94,95)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-94h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_150_05deg/GLM_array_150_17-18_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl94 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl94 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl94

In [ ]:
da_all_agregated = da.isel(time=slice(95,96)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-95h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_150_05deg/GLM_array_150_16-17_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl95 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl95 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl95

In [ ]:
da_all_agregated = da.isel(time=slice(96,97)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-96h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_150_05deg/GLM_array_150_15-16_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl96 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl96 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl96

In [ ]:
da_all_agregated = da.isel(time=slice(97,98)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-97h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_150_05deg/GLM_array_150_14-15_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl97 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl97 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl97

In [ ]:
da_all_agregated = da.isel(time=slice(98,99)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-98h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_150_05deg/GLM_array_150_13-14_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl98 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl98 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl98

In [ ]:
da_all_agregated = da.isel(time=slice(99,100)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-99h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_150_05deg/GLM_array_150_12-13_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl99 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl99 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl99

In [ ]:
da_all_agregated = da.isel(time=slice(100,101)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-100h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_150_05deg/GLM_array_150_11-12_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl100 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl100 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl100

In [ ]:
da_all_agregated = da.isel(time=slice(101,102)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-101h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_150_05deg/GLM_array_150_10-11_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl101 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl101 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl101

In [ ]:
da_all_agregated = da.isel(time=slice(102,103)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-102h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_150_05deg/GLM_array_150_09-10_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl102 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl102 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl102

In [ ]:
da_all_agregated = da.isel(time=slice(103,104)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-103h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_150_05deg/GLM_array_150_08-09_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl103 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl103 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl103

In [ ]:
da_all_agregated = da.isel(time=slice(104,105)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-104h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_150_05deg/GLM_array_150_07-08_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl104 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl104 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl104

In [ ]:
da_all_agregated = da.isel(time=slice(105,106)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-105h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_150_05deg/GLM_array_150_06-07_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl105 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl105 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl105

In [ ]:
da_all_agregated = da.isel(time=slice(106,107)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-106h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_150_05deg/GLM_array_150_05-06_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl106 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl106 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl106

In [ ]:
da_all_agregated = da.isel(time=slice(107,108)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-107h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_150_05deg/GLM_array_150_04-05_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl107 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl107 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl107

In [ ]:
da_all_agregated = da.isel(time=slice(108,109)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-108h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_150_05deg/GLM_array_150_03-04_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl108 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl108 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl108

In [ ]:
da_all_agregated = da.isel(time=slice(109,110)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-109h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_150_05deg/GLM_array_150_02-03_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl109 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl109 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl109

In [ ]:
da_all_agregated = da.isel(time=slice(110,111)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-110h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_150_05deg/GLM_array_150_01-02_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl110 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl110 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl110

In [ ]:
da_all_agregated = da.isel(time=slice(111,112)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-111h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_150_05deg/GLM_array_150_00-01_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl111 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl111 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl111

In [ ]:
da_all_agregated = da.isel(time=slice(112,113)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-112h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_149_05deg/GLM_array_149_23-00_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl112 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl112 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl112

In [ ]:
da_all_agregated = da.isel(time=slice(113,114)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-113h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_149_05deg/GLM_array_149_22-23_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl113 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl113 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl113

In [ ]:
da_all_agregated = da.isel(time=slice(114,115)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-114h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_149_05deg/GLM_array_149_21-22_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl114 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl114 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl114

In [ ]:
da_all_agregated = da.isel(time=slice(115,116)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-115h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_149_05deg/GLM_array_149_20-21_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl115 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl115 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl115

In [ ]:
da_all_agregated = da.isel(time=slice(116,117)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-116h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_149_05deg/GLM_array_149_19-20_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl116 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl116 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl116

In [ ]:
da_all_agregated = da.isel(time=slice(117,118)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-117h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_149_05deg/GLM_array_149_18-19_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl117 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl117 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl117

In [ ]:
da_all_agregated = da.isel(time=slice(118,119)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-118h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_149_05deg/GLM_array_149_17-18_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl118 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl118 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl118

In [ ]:
da_all_agregated = da.isel(time=slice(119,120)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-119h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_149_05deg/GLM_array_149_16-17_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl119 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl119 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl119

In [ ]:
da_all_agregated = da.isel(time=slice(120,121)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-120h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_149_05deg/GLM_array_149_15-16_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl120 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl120 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl120

In [ ]:
da_all_agregated = da.isel(time=slice(121,122)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-121h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_149_05deg/GLM_array_149_14-15_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl121 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl121 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl121

In [ ]:
da_all_agregated = da.isel(time=slice(122,123)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-122h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_149_05deg/GLM_array_149_13-14_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl122 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl122 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl122

In [ ]:
da_all_agregated = da.isel(time=slice(123,124)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-123h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_149_05deg/GLM_array_149_12-13_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl123 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl123 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl123

In [ ]:
da_all_agregated = da.isel(time=slice(124,125)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-124h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_149_05deg/GLM_array_149_11-12_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl124 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl124 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl124

In [ ]:
da_all_agregated = da.isel(time=slice(125,126)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-125h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_149_05deg/GLM_array_149_10-11_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl125 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl125 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl125

In [ ]:
da_all_agregated = da.isel(time=slice(126,127)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-126h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_149_05deg/GLM_array_149_09-10_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl126 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl126 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl126

In [ ]:
da_all_agregated = da.isel(time=slice(127,128)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-127h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_149_05deg/GLM_array_149_08-09_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl127 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl127 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl127

In [ ]:
da_all_agregated = da.isel(time=slice(128,129)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-128h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_149_05deg/GLM_array_149_07-08_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl128 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl128 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl128

In [ ]:
da_all_agregated = da.isel(time=slice(129,130)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-129h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_149_05deg/GLM_array_149_06-07_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl129 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl129 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl129

In [ ]:
da_all_agregated = da.isel(time=slice(130,131)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-130h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_149_05deg/GLM_array_149_05-06_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl130 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl130 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl130

In [ ]:
da_all_agregated = da.isel(time=slice(131,132)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-131h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_149_05deg/GLM_array_149_04-05_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl131 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl131 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl131

In [ ]:
da_all_agregated = da.isel(time=slice(132,133)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-132h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_149_05deg/GLM_array_149_03-04_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl132 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl132 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl132

In [ ]:
da_all_agregated = da.isel(time=slice(133,134)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-133h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_149_05deg/GLM_array_149_02-03_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl133 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl133 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl133

In [ ]:
da_all_agregated = da.isel(time=slice(134,135)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-134h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_149_05deg/GLM_array_149_01-02_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl134 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl134 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl134

In [ ]:
da_all_agregated = da.isel(time=slice(135,136)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-135h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_149_05deg/GLM_array_149_00-01_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl135 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl135 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl135

In [ ]:
da_all_agregated = da.isel(time=slice(136,137)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-136h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_148_05deg/GLM_array_148_23-00_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl136 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl136 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl136

In [ ]:
da_all_agregated = da.isel(time=slice(137,138)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-137h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_148_05deg/GLM_array_148_22-23_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl137 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl137 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl137

In [ ]:
da_all_agregated = da.isel(time=slice(138,139)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-138h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_148_05deg/GLM_array_148_21-22_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl138 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl138 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl138

In [ ]:
da_all_agregated = da.isel(time=slice(139,140)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-139h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_148_05deg/GLM_array_148_20-21_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl139 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl139 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl139

In [ ]:
da_all_agregated = da.isel(time=slice(140,141)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-140h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_148_05deg/GLM_array_148_19-20_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl140 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl140 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl140

In [ ]:
da_all_agregated = da.isel(time=slice(141,142)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-141h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_148_05deg/GLM_array_148_18-19_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl141 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl141 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl141

In [ ]:
da_all_agregated = da.isel(time=slice(142,143)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-142h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_148_05deg/GLM_array_148_17-18_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl142 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl142 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl142

In [ ]:
da_all_agregated = da.isel(time=slice(143,144)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-143h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_148_05deg/GLM_array_148_16-17_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl143 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl143 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl143

In [ ]:
da_all_agregated = da.isel(time=slice(144,145)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-144h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_148_05deg/GLM_array_148_15-16_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl144 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl144 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl144

In [ ]:
da_all_agregated = da.isel(time=slice(145,146)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-145h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_148_05deg/GLM_array_148_14-15_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl145 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl145 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl145

In [ ]:
da_all_agregated = da.isel(time=slice(146,147)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-146h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_148_05deg/GLM_array_148_13-14_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl146 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl146 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl146

In [ ]:
da_all_agregated = da.isel(time=slice(147,148)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-147h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_148_05deg/GLM_array_148_12-13_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl147 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl147 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl147

In [ ]:
da_all_agregated = da.isel(time=slice(148,149)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-148h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_148_05deg/GLM_array_148_11-12_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl148 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl148 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl148

In [ ]:
da_all_agregated = da.isel(time=slice(149,150)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-149h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_148_05deg/GLM_array_148_10-11_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl149 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl149 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl149

In [ ]:
da_all_agregated = da.isel(time=slice(150,151)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-150h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_148_05deg/GLM_array_148_09-10_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl150 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl150 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl150

In [ ]:
da_all_agregated = da.isel(time=slice(151,152)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-151h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_148_05deg/GLM_array_148_08-09_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl151 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl151 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl151

In [ ]:
da_all_agregated = da.isel(time=slice(152,153)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-152h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_148_05deg/GLM_array_148_07-08_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl152 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl152 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl152

In [ ]:
da_all_agregated = da.isel(time=slice(153,154)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-153h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_148_05deg/GLM_array_148_06-07_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl153 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl153 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl153

In [ ]:
da_all_agregated = da.isel(time=slice(154,155)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-154h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_148_05deg/GLM_array_148_05-06_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl154 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl154 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl154

In [ ]:
da_all_agregated = da.isel(time=slice(155,156)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-155h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_148_05deg/GLM_array_148_04-05_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl155 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl155 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl155

In [ ]:
da_all_agregated = da.isel(time=slice(156,157)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='t-156h',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Res. Time(s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_148_05deg/GLM_array_148_03-04_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl156 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl156 * bogota * flight_path * first_rel * second_rel

In [ ]:
pl * pl156

In [ ]:
da_all_agregated = da.isel(time=slice(11,33)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='Max residence time t-11 --> -33h all altitudes',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Max Residence Time (s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl

In [ ]:
pl * pl11 * pl12 * pl13 * pl14 * pl15 * pl16 * pl17 * pl18 * pl19 * pl20 * pl21 * pl22 * pl23 * pl24 * pl25 * pl26 * pl27 * pl28 * pl29 * pl30 * pl31 * pl32

In [ ]:
da_all_agregated = da.isel(time=slice(34,48)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='Max residence time t-34 --> -48h all altitudes',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Max Residence Time (s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl

In [ ]:
pl * pl34 * pl35 * pl36 * pl37 * pl38 * pl39 * pl40 * pl41 * pl42 * pl43 * pl44 * pl46 * pl47 * pl48

In [ ]:
da_all_agregated = da.isel(time=slice(49,60)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='Max residence time t-49 --> -60h all altitudes',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Max Residence Time (s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl

In [ ]:
pl * pl49 * pl50 * pl51 * pl52 * pl53 * pl54 * pl55 * pl56 * pl57 * pl58 * pl59 * pl60 

In [ ]:
da_all_agregated = da.isel(time=slice(61,72)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='Max residence time t-61 --> -72h all altitudes',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Max Residence Time (s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl

In [ ]:
pl * pl61 * pl62 * pl63 * pl64 * pl65 * pl66 * pl67 * pl68  * pl70 * pl71 * pl72

In [ ]:
da_all_agregated = da.isel(time=slice(72,84)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='Max residence time t-72 --> -84h all altitudes',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Max Residence Time (s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl

In [ ]:
pl * pl72 * pl72 * pl74 * pl75 * pl76 * pl77 * pl78 * pl79  * pl80 * pl81 * pl82 * pl83 * pl84

In [ ]:
da_all_agregated = da.isel(time=slice(84,96)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='Max residence time t-84 --> -96h all altitudes',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Max Residence Time (s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl

In [ ]:
pl * pl84 * pl85 * pl86 * pl87 * pl88 * pl89 * pl90 * pl94 * pl95 * pl96

In [ ]:
da_all_agregated = da.isel(time=slice(96,108)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='Max residence time t-96 --> -108h all altitudes',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Max Residence Time (s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl

In [ ]:
pl * pl96 * pl97 * pl98 * pl99 * pl100 * pl101 * pl102 * pl103 * pl104 * pl105 * pl106 * pl107 * pl108

In [ ]:
da_all_agregated = da.isel(time=slice(108,120)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='Max residence time t-108 --> -120h all altitudes',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Max Residence Time (s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl

In [ ]:
pl * pl108 * pl109 * pl110 * pl111 * pl112 * pl113 * pl114 * pl115 * pl116 * pl117 * pl118 * pl119 * pl120

In [ ]:
da_all_agregated = da.isel(time=slice(120,132)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='Max residence time t-120 --> -132h all altitudes',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Max Residence Time (s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl

In [ ]:
pl * pl120 * pl121 * pl122 * pl123 * pl124 * pl125 * pl126 * pl127 * pl128 * pl129 * pl130 * pl131 * pl132

In [ ]:
da_all_agregated = da.isel(time=slice(132,144)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='Max residence time t-132 --> -144h all altitudes',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Max Residence Time (s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl

In [ ]:
pl * pl132 * pl133 * pl134 * pl135 * pl136 * pl137 * pl138 * pl139 * pl140 * pl141 * pl142 * pl143 * pl144

In [ ]:
da_all_agregated = da.isel(time=slice(144,156)).sum(dim=('height', 'time', 'pointspec'))
pl = da_all_agregated.hvplot.image(
    x='longitude', y='latitude',  title='Max residence time t-144 --> -156h all altitudes',
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Max Residence Time (s)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='coolwarm',
)
pl

In [ ]:
pl * pl144 * pl145 * pl146 * pl147 * pl148 * pl149 * pl150 #* pl151 * pl152 * pl153 * pl154 * pl155 * pl156

In [ ]:
pl * pl144 * pl145 * pl146 * pl147 * pl148 * pl149 * pl150 * pl151 * pl152 * pl153 * pl154 * pl155 * pl156

In [ ]:
# comparison flash/group 1°, 1h

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_148/GLM_array_148_04-05_batch_bis.nc')
flash_en = dr.num_flash.where(dr.flash_energy > 0)
flash_num = dr.num_flash.where(dr.num_flash > 0)

pl155 = (flash_en).hvplot.image(
    x='longitude', y='latitude',  title='Total flash energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Flash energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='spring'
    #cmap='YlOrRd'
)
pl155 * bogota * flight_path * first_rel * second_rel

In [ ]:
dr = xr.open_dataset('/o3p/macc/test/GLM_array_148_group/GLM_array_148_04-05_batch_bis.nc')
group_en = dr.num_group.where(dr.group_energy > 0)
group_num = dr.num_group.where(dr.num_group > 0)

pl155g = (group_en).hvplot.image(
    x='longitude', y='latitude',  title='Total group energy', 
    xlabel = 'Longitude (°)', ylabel = 'Latitude (°)', clabel = 'Group energy (J)',
    frame_height=350, 
    projection=ccrs.PlateCarree(), 
    geo=True, project=False, #global_extent=True,
    xlim=(-140, -20), ylim=(-60, 60), 
    framewise=False, coastline=True, dynamic=True,
    cmap='ocean'
    #cmap='YlOrRd'
)
pl155g * bogota * flight_path * first_rel * second_rel

In [ ]:
pl155 + pl155g 

In [ ]:
pl155 * pl155g 

In [ ]:
pl155g * pl155 